In [2]:
using Random
using Distributions
using FLoops
using Base.Threads

include("data.jl")
include("util.jl")

include("intervals/permutation.jl")
include("intervals/bootstrap.jl")
include("intervals/t.jl")

Threads.nthreads()

16

In [3]:
"""
DATA CONFIG
"""
alpha = 0.05

# data
B  = 100    # num. coverage probabilities per boxplot
S  = 1800   # num. samples per coverage probability
nx = 8      # size of group 1
ny = 8     # size of group 2
px, py = partition(nx, ny)
mc_size = try
    (binomial(nx+ny, nx) > 45_000) ? 10_000 : 0
catch
    0
end

dtype = Float32
seed = 123

"""
POPULATION SETTINGS
"""

Random.seed!(seed)
distrTypeX = Normal{dtype}
muX = dtype.(round.(rand(Uniform(0, 1), B), digits=4))
sdX = 2 * muX
distrX = map(distrTypeX, muX, sdX)

distrTypeY = Normal{dtype}
muY = dtype.(round.(rand(Uniform(0, 1), B), digits=4))
sdY = 2 * muY
distrY = map(distrTypeY, muY, sdY)

@show distrY[1:3]

deltas = @. mean(distrX) - mean(distrY);

distrY[1:3] = Normal{Float32}[Normal{Float32}(μ=0.9542f0, σ=1.9084f0), Normal{Float32}(μ=0.8459f0, σ=1.6918f0), Normal{Float32}(μ=0.5867f0, σ=1.1734f0)]


In [4]:
Random.seed!(123)
xs = [rand(distrX[i], nx, S) for i in 1:B]
ys = [rand(distrY[i], nx, S) for i in 1:B]
ys[2]

8×1800 Matrix{Float32}:
  1.64304    0.427069   4.01956   …   2.63878    2.70283    -0.202007
  0.418447   1.34317   -1.46597      -0.545218   3.14764     2.42861
  1.74494    0.55856    0.130721     -3.98272    2.51431    -1.66733
 -3.92348    3.17468    0.763631      0.754839  -0.0685847   2.66572
 -0.386198   3.04206   -1.60965       0.773651  -1.56575    -1.44321
  4.09075   -0.471276   2.1126    …   4.67817    1.68657    -1.48642
  1.38358    1.98887    2.30283       0.439505  -1.81795     1.48651
  0.373906   1.98244   -0.554305     -0.532434   1.47835     1.15004

In [5]:
X = reshape(hcat(xs...), 8, S, B)
Y = reshape(hcat(ys...), 8, S, B)
Y[:,:,2]

8×1800 Matrix{Float32}:
  1.64304    0.427069   4.01956   …   2.63878    2.70283    -0.202007
  0.418447   1.34317   -1.46597      -0.545218   3.14764     2.42861
  1.74494    0.55856    0.130721     -3.98272    2.51431    -1.66733
 -3.92348    3.17468    0.763631      0.754839  -0.0685847   2.66572
 -0.386198   3.04206   -1.60965       0.773651  -1.56575    -1.44321
  4.09075   -0.471276   2.1126    …   4.67817    1.68657    -1.48642
  1.38358    1.98887    2.30283       0.439505  -1.81795     1.48651
  0.373906   1.98244   -0.554305     -0.532434   1.47835     1.15004

In [13]:
B = 100
S = 10

T = Threads.nthreads()
results = Array{Union{Nothing, Tuple}, 3}(undef, 6, B, S)
fill!(results, nothing)

#@time Threads.@threads for (i,j) in collect(Iterators.product(1:B, 1:S)) # 15.52 sec on (B,S) = (5, 1800)
@time @floop ThreadedEx(basesize=div(B*S, T)) for b in 1:B, s in 1:S
    @inbounds x = X[:,s,b]
    @inbounds y = Y[:,s,b]
    
    results[1, b, s] = permInterval(x, y, deltas[b], (px, py, mc_size), true, 0.05, twoSided, twoSided)
    results[2, b, s] = permInterval(x, y, deltas[b], (px, py, mc_size), false, alpha, twoSided, twoSided)
    results[3, b, s] = permInterval(x, y, deltas[b], (px, py, mc_size), true, alpha/2, greater, smaller)
    results[4, b, s] = permInterval(x, y, deltas[b], (px, py, mc_size), false, alpha/2, greater, smaller)
    
    """
    lo, hi = bootstrap(x, y, 0.05, false, 10_000)
    coverages[5] += (lo <= delta <= hi)

    lo, hi = tconf(x, y, 0.05, false)
    coverages[6] += (lo <= delta <= hi)
    """
end

i = 1
per_method = []
for is_two_sided in [true, false]
    for pooled in [true, false]
        averages = []
        
        for row in eachrow(results[i,:,:])
            coverage = sum([j for (j, _) in row]) / S
            width = sum([j for (_, j) in row]) / S
            push!(averages, (coverage, width))
        end
        push!(per_method, averages)
        
        alpha_ = is_two_sided ? alpha : alpha / 2
        save(averages, distrX, distrY, alpha_, pooled, is_two_sided)
        i += 1
    end
end

  7.235667 seconds (6.48 M allocations: 69.601 GiB, 11.87% gc time, 1.57% compilation time)


4-element Vector{Any}:
 Any[(0.9, 3.427311873339935), (0.9, 2.394524247724658), (0.8, 2.1974981228157184), (1.0, 1.3356003101044496), (0.8, 0.7358609112716937), (0.9, 0.832752014937228), (0.9, 1.3569443539703812), (0.9, 2.990785960701275), (0.9, 0.39854449124318553), (1.0, 1.9772253658628631)  …  (1.0, 1.1516825105566277), (0.9, 2.268885293866494), (1.0, 2.319162144572429), (0.9, 2.631759669909701), (1.0, 0.36326906067069664), (1.0, 1.7441511597468256), (0.9, 2.0780659791263947), (0.9, 2.1327812488110696), (0.8, 0.3811564452536488), (0.9, 2.4128481089222458)]
 Any[(0.9, 3.412378722741748), (0.9, 2.3938926152039963), (0.8, 2.1891448965453186), (1.0, 1.564556309192551), (0.8, 0.7349399738857878), (0.9, 0.8336170113681746), (0.9, 1.3559603346243345), (0.8, 2.9682378485234837), (0.9, 0.39746973222672666), (1.0, 1.9771481343213442)  …  (1.0, 1.2117820158624182), (1.0, 2.2513591915541005), (1.0, 2.3082002358986857), (0.9, 2.6019247454712846), (1.0, 0.3610296505975924), (1.0, 1.75579264524615

Alex Han

InvertingPermutationTests2022